In [1]:
#@title Βιβλιοθήκες.
#!pip install --upgrade torch
#!pip install pytorchtools

import pandas as pd

import shutil

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MultiLabelBinarizer

from google.colab import drive
from google.colab import files
from google.colab import output

import pprint, pickle

import sys
import gc

from PIL import Image
from __future__ import print_function, division
import os
import torch
from skimage import io, transform
import matplotlib.pyplot as plt
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

from torchvision.transforms.transforms import ToTensor
import os
import argparse

from sklearn import metrics
from tqdm import tqdm
import torch.utils.data
import torchvision.transforms as transforms
from torchvision.datasets.folder import pil_loader


In [2]:
#@title Δεδομένα στο Drive.
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [28]:
class Artists(Dataset):
    def __init__(self, base_path, image_ids_fn, images_dir):
        self.base_path = base_path
        self.image_ids_fn = image_ids_fn
        self.images_dir = images_dir
        with open(base_path + image_ids_fn, 'r') as fp:
            rows = list(fp)
            self.fnames = [s.strip().split(',')[0] for s in rows[1:]]
            self.img_class_ids = [int(s.strip().split(',')[1]) for s in rows[1:]]
            self.img_ids = list(range(len(self.fnames)))
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize((0.5243, 0.4849, 0.4273),(0.2744, 0.2674, 0.2680))
        ])

    
    def __getitem__(self, index):
        img_fname = self.fnames[index]
        image = pil_loader(self.base_path + self.images_dir + img_fname)

        if self.transform is not None:
            image = self.transform(image)
        
        return image, self.img_class_ids[index]

    def __len__(self):
        return len(self.img_ids)

In [32]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

data_fn = "gdrive/MyDrive/art_recognition/Datasets/Dataset 100/"
dataset = Artists(data_fn, "val_100.csv", "/Data/")
loader = torch.utils.data.DataLoader(
    dataset,
    shuffle=True,
    batch_size=8,
    pin_memory=True,
    num_workers=0,
    # drop_last=True,
)

In [33]:
Image.MAX_IMAGE_PIXELS = None

datasets = list()
for i in dataset:
  datasets.append(i)

In [34]:
output = open('val_100.pkl', 'wb')
pickle.dump(datasets, output)
output.close()
!cp val_100.pkl '/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/'

In [23]:
f = open("/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/train_100.pkl","rb")
li = pickle.load(f)

# Normalization

In [ ]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std
print(get_mean_and_std(loader))

(tensor([0.5243, 0.4849, 0.4273]), tensor([0.2744, 0.2674, 0.2680]))
